## Install the Kubeflow-metadata library¶

In [4]:
!pip install kubeflow-metadata --user
!pip install pandas --user
!pip install tornado>=6.0.3 --user
!pip install kubeflow-fairing --user
!pip install scikit-learn --user
!pip install kfserving

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable


In [15]:
!kubectl get pods

NAME         READY   STATUS    RESTARTS   AGE
example2-0   2/2     Running   0          13m


In [ ]:
import pandas
from kubeflow.metadata import metadata
from datetime import datetime
from uuid import uuid4
import logging
from google.cloud import storage

In [ ]:
import google.cloud as gcloud
dir(gcloud)

In [ ]:
METADATA_STORE_HOST = "metadata-grpc-service.kubeflow" # default DNS of Kubeflow Metadata gRPC serivce.
METADATA_STORE_PORT = 8080

## Create a workspace

In [ ]:
ws1 = metadata.Workspace(
    # Connect to metadata service in namespace kubeflow in k8s cluster.
    store=metadata.Store(grpc_host=METADATA_STORE_HOST, grpc_port=METADATA_STORE_PORT),
    name="workspace_1",
    description="a workspace for testing",
    labels={"n1": "v1"})
pandas.DataFrame.from_dict(ws1.list())

In [ ]:
storage_client = storage.Client()
bucket_name = 'mlops-282422-kubeflowpipelines-default'
model_path = 'mockup-model.dat'
model_file = joblib.load(model_path)
model_bucket = storage_client.get_bucket(bucket_name)
model_blob = model_bucket.blob(model_path)
model_blob.upload_from_filename(model_path)

In [22]:
!kubectl get pods -o wide

NAME                                 READY   STATUS      RESTARTS   AGE    IP            NODE                                                  NOMINATED NODE   READINESS GATES
example2-0                           2/2     Running     0          135m   10.60.0.81    gke-kubeflow-dev-kubeflow-dev-cpu-poo-747b81cc-pk7s   <none>           <none>
fairing-builder-4cn2b-jswbs          0/1     Completed   0          90m    10.60.1.102   gke-kubeflow-dev-kubeflow-dev-cpu-poo-747b81cc-qp4l   <none>           <none>
fairing-builder-mq7qg-c2jgz          0/1     Completed   0          73m    10.60.1.103   gke-kubeflow-dev-kubeflow-dev-cpu-poo-747b81cc-qp4l   <none>           <none>
mnist-model-c54dbf959-drxl2          1/1     Running     0          38s    10.60.0.89    gke-kubeflow-dev-kubeflow-dev-cpu-poo-747b81cc-pk7s   <none>           <none>
mnist-tensorboard-68fc7f4d89-4kr9f   2/2     Running     0          53m    10.60.1.110   gke-kubeflow-dev-kubeflow-dev-cpu-poo-747b81cc-qp4l   <none>       

In [ ]:
model_bucket.get_blob('mockup-model.dat')#.download_to_filename('model.dat')
joblib.load('model.dat')

In [ ]:
for i in model_bucket.list_blobs(): print(i.name)

In [ ]:
# fairing:include-cell
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

def read_synthetic_input(test_size=0.25):
    """generate synthetic data and split it into train and test."""
    # generate regression dataset
    X, y = make_regression(n_samples=200, n_features=5, noise=0.1)
    train_X, test_X, train_y, test_y = train_test_split(X,
                                                      y,
                                                      test_size=test_size,
                                                      shuffle=False)

    imputer = SimpleImputer()
    train_X = imputer.fit_transform(train_X)
    test_X = imputer.transform(test_X)

    return (train_X, train_y), (test_X, test_y)


In [ ]:
from sklearn.base import BaseEstimator
import joblib
import os
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error

class PersistenteModel:
    def __init__(self,  workspace, model_file=None):
        if not model_file:
            if "MODEL_FILE" in os.environ:
                print("model_file not supplied; checking environment variable")
                model_file = os.getenv("MODEL_FILE")
            else:
                print("model_file not supplied; using the default")
                model_file = "mockup-model.dat"
        self.__model_file = model_file
        print("model_file={0}".format(self.__model_file))
        self.__workspace = workspace
        self.exec = self.create_execution()
    
    @property
    def model_file(self):
        return self.__model_file
    
    @property
    def workspace(self):
        return self.__workspace

    def create_execution(self):
        pass

    def _save_model(self, model):
        """Save model for serving."""
        joblib.dump(model, self.__model_file)
        logging.info("Model export success: %s", self.__model_file)
        
    def _load_model(self):
        print('Loading model from {}'.format(self.__model_file))
        return joblib.load(self.__model_file)
        
    def create_execution(self):                
        r = metadata.Run(
            workspace=self.workspace,
            name="example-synthetic-faring-run" + datetime.utcnow().isoformat("T"),
            description="a notebook run")

        return metadata.Execution(
            name = "execution" + datetime.utcnow().isoformat("T"),
            workspace=self.workspace,
            run=r,
            description="execution for training example-synthetic")


class DecisionTreeModel(BaseEstimator, PersistenteModel):
    
    def __init__(self, workspace, model_file=None):
        super().__init__(workspace=workspace, model_file=model_file)
        self.model = None
    
    def fit(self, X, y):
        self.model = DecisionTreeRegressor().fit(X, y)
        self._save_model(self.model)
        """
        self.exec.log_output(metadata.Model(
            name="example-model",
            description="Model using synthetic data",
            owner="veragua.alb@gmail.com",
            uri=self.model_file,
            model_type="decisionTree_regression",
            training_framework={
                "name": "decisionTree",
                "version": "0.0.1"
            },
            hyperparameters={},
            version=datetime.utcnow().isoformat("T")))
        """
        return self
    
    def pred_eval(self, X, y):
        y_pred = self.predict(X)
        print(mean_absolute_error(y_pred, y))
        return y_pred

    def predict(self, X):
        if not self.model:
            self.model = self._load_model()
        return self.model.predict(X)


(train_X, train_y), (test_X, test_y) = read_synthetic_input()
model = DecisionTreeModel(ws1).fit(train_X, train_y)

In [ ]:
from kubeflow.fairing import constants
constants.constants.KANIKO_IMAGE = "gcr.io/kaniko-project/executor:v0.14.0"

In [ ]:
def load_model(model_path):
  local_model_path = model_path
  if model_path.startswith("gs://"):
    gcs_path = model_path
    train_bucket_name, train_path = split_gcs_uri(gcs_path)

    storage_client = storage.Client()
    train_bucket = storage_client.get_bucket(train_bucket_name)
    train_blob = train_bucket.blob(train_path)

    local_model_path = "/tmp/model.dat"
    logging.info("Downloading model to %s", local_model_path)
    train_blob.download_to_filename(local_model_path)

  model = joblib.load(local_model_path)
  return model
load_model('gcs://my-bucket/mnist')

## Create a run in a workspace


In [ ]:
r = metadata.Run(
    workspace=ws1,
    name="run-" + datetime.utcnow().isoformat("T") ,
    description="a run in ws_1",
)

## Create an execution in a run

In [ ]:
exec = metadata.Execution(
    name = "execution" + datetime.utcnow().isoformat("T") ,
    workspace=ws1,
    run=r,
    description="execution example",
)
print("An execution was created with id %s" % exec.id)


## Log a data set

In [ ]:
date_set_version = "data_set_version_" + str(uuid4())
data_set = exec.log_input(
        metadata.DataSet(
            description="an example data",
            name="mytable-dump",
            owner="owner@my-company.org",
            uri="file://path/to/dataset",
            version=date_set_version,
            query="SELECT * FROM mytable"))
print("Data set id is {0.id} with version '{0.version}'".format(data_set))

## Log a model

In [ ]:
model_version = "model_version_" + str(uuid4())
model = exec.log_output(
    metadata.Model(
            name="MNIST",
            description="model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist",
            model_type="neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v1.0"
            },
            hyperparameters={
                "learning_rate": 0.5,
                "layers": [10, 3, 1],
                "early_stop": True
            },
            version=model_version,
            labels={"mylabel": "l1"}))
print(model)
print("\nModel id is {0.id} and version is {0.version}".format(model))


## Log the evaluation of a model

In [ ]:
metrics = exec.log_output(
    metadata.Metrics(
            name="MNIST-evaluation",
            description="validating the MNIST model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist-eval.csv",
            data_set_id=str(data_set.id),
            model_id=str(model.id),
            metrics_type=metadata.Metrics.VALIDATION,
            values={"accuracy": 0.95},
            labels={"mylabel": "l1"}))
print("Metrics id is %s" % metrics.id)

## Add Metadata for serving the model

In [ ]:
serving_application = metadata.Execution(
    name="serving model",
    workspace=ws1,
    description="an execution to represent model serving component",
)
# Noticed we use model name, version, uri to uniquely identify existing model.
served_model = metadata.Model(
    name="MNIST",
    uri="gcs://my-bucket/mnist",
    version=model.version,
)
m=serving_application.log_input(served_model)
print("Found the mode with id {0.id} and version '{0.version}'.".format(m))

## List all models in the workspace

## Basic Lineage Tracking

In [ ]:
print("Model id is %s\n" % model.id)
    
model_events = ws1.store.get_events_by_artifact_ids([model.id])

execution_ids = set(e.execution_id for e in model_events)
print("All executions related to the model are {}".format(execution_ids))
# assert execution_ids == set([serving_application.id, exec.id])

trainer_events = ws1.store.get_events_by_execution_ids([exec.id])
artifact_ids = set(e.artifact_id for e in trainer_events)
print("All artifacts related to the training event are {}".format(artifact_ids))# assert artifact_ids == set([model.id, metrics.id, data_set.id])

# Tensorflow tutorial

1. Before executions can be recorded, ArtifactTypes have to be registered.

In [ ]:
from ml_metadata import metadata_store
# from ml_metadata.proto import metadata_store_pb2
store = ws1.store
metadata_store_pb2 = metadata.mlpb

In [ ]:
# Create ArtifactTypes, e.g., Data and Model
data_type = metadata_store_pb2.ArtifactType()
data_type.name = "DataSet"
data_type.properties["day"] = metadata_store_pb2.INT
data_type.properties["split"] = metadata_store_pb2.STRING
data_type_id = store.put_artifact_type(data_type)
data_type

In [ ]:
model_type = metadata_store_pb2.ArtifactType()
model_type.name = "SavedModel"
model_type.properties["version"] = metadata_store_pb2.INT
model_type.properties["name"] = metadata_store_pb2.STRING
model_type_id = store.put_artifact_type(model_type)
model_type

2. Before executions can be recorded, ExecutionTypes have to be registered for all steps in our ML workflow.

In [ ]:
# Create ExecutionType, e.g., Trainer
trainer_type = metadata_store_pb2.ExecutionType()
trainer_type.name = "Trainer"
trainer_type.properties["state"] = metadata_store_pb2.STRING
trainer_type_id = store.put_execution_type(trainer_type)
trainer_type

3. Once types are registered, we create a DataSet Artifact.

In [ ]:
# Declare input artifact of type DataSet
data_artifact = metadata_store_pb2.Artifact()
data_artifact.uri = 'path/to/data'
data_artifact.properties["day"].int_value = 1
data_artifact.properties["split"].string_value = 'train'
data_artifact.type_id = data_type_id
data_artifact_id = store.put_artifacts([data_artifact])[0]
data_artifact

4. With the DataSet Artifact created, we can create the Execution for a Trainer run

## Miscelanuous

In [ ]:
import os
from oauth2client.client import GoogleCredentials
credentials = GoogleCredentials.get_application_default()

In [ ]:
model.pred_eval(test_X, test_y)

In [ ]:
from kubeflow import fairing   
from kubeflow.fairing.builders import append
from kubeflow.fairing.deployers import job
from kubeflow.fairing.preprocessors.converted_notebook import ConvertNotebookPreprocessorWithFire
GCP_PROJECT = fairing.cloud.gcp.guess_project_name()
DOCKER_REGISTRY = 'gcr.io/{}/fairing-job'.format(GCP_PROJECT)
DOCKER_REGISTRY

In [ ]:
dir(kubeflow)

In [ ]:
!kubectl get jobs

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

x = tf.placeholder(tf.float32, [None, 784])

W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

y = tf.nn.softmax(tf.matmul(x, W) + b)

y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

train_step = tf.train.GradientDescentOptimizer(0.05).minimize(cross_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for _ in range(1000):
  batch_xs, batch_ys = mnlist.train.next_batch(100)
  sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy: ", sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))